In [ ]:
import requests as r
from bs4 import BeautifulSoup
import pandas as pd
import re

In [ ]:
version = r.get('https://ddragon.leagueoflegends.com/api/versions.json').json()[0]
champions = r.get('http://ddragon.leagueoflegends.com/cdn/15.4.1/data/en_US/champion.json').json()['data']
name_of_champions = [champions[champion]['name'] for champion in champions]

In [ ]:
for champion in name_of_champions:
  html = r.get(f"https://leagueoflegends.fandom.com/pt-br/wiki/{champion}/LoL/Áudio")
  if html.status_code != 200:
    name_of_champions.remove(champion)

In [ ]:
csv = []

for champion in name_of_champions:

  html = r.get(f"https://leagueoflegends.fandom.com/pt-br/wiki/{champion}/LoL/%C3%81udio")
  soup = BeautifulSoup(html.text, 'html.parser')

  unsorted_lines = soup.find('div', class_="mw-parser-output")

  if unsorted_lines is None:
    continue

  unsorted_lines = unsorted_lines.find_all('ul')

  for ul in unsorted_lines:
    list_elements = ul.find_all('li')

    for element in list_elements:
      list_audio = element.find_all('audio')
      list_text = element.find_all('i')

      for audio in list_audio:
        link = audio.find('source')['src']

        for text in list_text:
          csv.append({'champion': champion, 'text': text, 'audio_link': link})

In [ ]:
pd.DataFrame(csv).to_csv('voicelines.csv', index=False)